In [1]:
import json, uuid, os
from datetime import datetime, timezone
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

if not uri or not database_name:
    raise RuntimeError("Missing MONGODB_URI or MONGODB_DATABASE")

# --- Connect to MongoDB ---
client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

# --- Enforce at most ONE is_empty=True via partial unique index ---
try:
    col.create_index(
        [("is_empty", 1)],
        unique=True,
        name="uniq_is_empty_true",
        partialFilterExpression={"is_empty": True},
        background=True,
    )
except Exception as e:
    # Index may already exist or fail to build in some environments; continue.
    pass

# --- Normalize: never allow missing items; keep is_empty in sync with items ---
col.update_many({"items": {"$exists": False}}, {"$set": {"items": []}})
col.update_many({"items": []}, {"$set": {"is_empty": True}})
col.update_many({"items": {"$exists": True, "$ne": []}}, {"$set": {"is_empty": False}})

# --- Reuse latest empty form if it exists (ensures we don't create a new one while any is_empty=True exists) ---
existing_empty = col.find_one({"is_empty": True}, sort=[("created", -1)])
if existing_empty:
    print("✅ Reusing latest EMPTY form (no new document created).")
    print(json.dumps(existing_empty, indent=2, default=str))
else:
    # --- Inherit some fields from latest document, if any ---
    latest_doc = col.find_one(sort=[("created", -1)])
    prev_qr = latest_doc.get("current_form_qr_code", "") if latest_doc else ""
    prev_tin = latest_doc.get("tin", "") if latest_doc else ""
    prev_location = latest_doc.get("location", "") if latest_doc else ""

    # --- Build new EMPTY form (items: [] and is_empty: True, never absent) ---
    now_utc = datetime.now(timezone.utc)
    new_doc = {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": prev_qr,
        "current_form_qr_code": now_utc.strftime("%Y%m%d"),  # short, sortable code
        "tin": prev_tin,
        "location": prev_location,
        "created": now_utc.isoformat(),
        "date": now_utc.date().isoformat(), 
        "is_empty": True,
    }

    try:
        res = col.insert_one(new_doc)
        new_doc["_id"] = str(res.inserted_id)
        print("✅ Created new EMPTY form (no existing empty one found).")
        print(json.dumps(new_doc, indent=2, default=str))
    except DuplicateKeyError:
        # Race condition: another writer inserted an is_empty=True doc first.
        existing_empty = col.find_one({"is_empty": True}, sort=[("created", -1)])
        print("♻️ Detected concurrent creator. Reusing the existing EMPTY form.")
        print(json.dumps(existing_empty, indent=2, default=str))


✅ Created new EMPTY form (no existing empty one found).
{
  "id": "730a24e8-2dca-4971-bb13-6c10134fa834",
  "previous_form_qr_code": "20251013",
  "current_form_qr_code": "20251013",
  "tin": "123-456-789",
  "location": "JEF Gas Station \u2013 Sikatuna Branch",
  "created": "2025-10-13T16:50:12.580820+00:00",
  "date": "2025-10-13",
  "is_empty": true,
  "_id": "68ed2dc43acb0c409bf17652"
}
